In [1]:
# Install Open CV
# Import cv2 and verify
# python3 -m pip install opencv-python
import cv2 as cv
import numpy as np
import chess
print(cv.__version__)

4.7.0


In [13]:
def display_matches(img, rects, x_incr, y_incr):
    """
    Display all potential matches of template
    """
    piece_notation = []

    for r in rects:
        print(r)
        cv.rectangle(img, (r[0], r[1]), (r[0] + r[2], r[1] + r[3]), (0, 0, 255), 2)
        x = int((r[0] + r[0] + r[2]) / 2)
        y = int((r[1] + r[1] + r[3]) / 2)
        cv.circle(img, (x,y), 10, (0, 0, 255), 2)

        notation = ""
        if x >= 0 and x <= x_incr:
            notation = notation + "a"
        elif x > x_incr and x <= 2*x_incr:
            notation = notation + "b"
        elif x > 2*x_incr and x <= 3*x_incr:
            notation = notation + "c"
        elif x > 3*x_incr and x <= 4*x_incr:
            notation = notation + "d"
        elif x > 4*x_incr and x <= 5*x_incr:
            notation = notation + "e"
        elif x > 5*x_incr and x <= 6*x_incr:
            notation = notation + "f"
        elif x > 6*x_incr and x <= 7*x_incr:
            notation = notation + "g"
        elif x > 7*x_incr and x <= 8*x_incr:
            notation = notation + "h"
        
        if x >= 0 and x <= y_incr:
            notation = notation + "8"
        elif x > y_incr and x <= 2*y_incr:
            notation = notation + "7"
        elif x > 2*y_incr and x <= 3*y_incr:
            notation = notation + "6"
        elif x > 3*y_incr and x <= 4*y_incr:
            notation = notation + "5"
        elif x > 4*y_incr and x <= 5*y_incr:
            notation = notation + "4"
        elif x > 5*y_incr and x <= 6*y_incr:
            notation = notation + "3"
        elif x > 6*y_incr and x <= 7*y_incr:
            notation = notation + "2"
        elif x > 7*y_incr and x <= 8*y_incr:
            notation = notation + "1"
        
        piece_notation.append(notation)
    return piece_notation
 



def check_color(img, temp, rect):
    """
    Determine if the piece is the same color as the given template
    Need to be very strict on bounds
    """
    y0, y1 = rect[1], rect[1] + rect[3]
    x0, x1 =  rect[0], rect[0] + rect[2]    
    template = (img[y0 : y1, x0 : x1]).copy()
    diff = cv.absdiff(temp, template)
    img_avg = cv.mean(diff)[0] / 255        
    return img_avg < 0.2

def find_template_multiple(img, temp):
    """
    Locate all matching templates
    """
    rects = []
    w, h = temp.shape[1], temp.shape[0]

    result = cv.matchTemplate(img, temp, cv.TM_CCOEFF_NORMED)
    threshold = 0.5 # matching threshold, relatively stable.
    loc = np.where( result >= threshold)

    for pt in zip(*loc[::-1]):
        rects.append((pt[0], pt[1], w, h))    

    # Non-max suppression
    rects, _ = cv.groupRectangles(rects, 1, 1)
    rects = [r for r in rects]   

    return rects

In [14]:
def _ (img_board, img_pieces):
    # Convert images to grayscale
    img_board_gray = cv.cvtColor(img_board, cv.COLOR_BGR2GRAY)
    img_pieces_gray = [cv.cvtColor(img_board, cv.COLOR_BGR2GRAY) for piece in img_pieces]
    
    # Morphological Erosion for stabilization
    s = 3
    kernel = np.ones((s, s), np.uint8)
    img_board_gray_erode = cv.erode(img_board_gray,kernel,iterations = 1)
    img_pieces_gray_erode = [cv.erode(img_piece_gray, kernel, iterations = 1) for img_piece_gray in img_pieces_gray]

    # All instances of matching templates
    matching_rects = [find_template_multiple(img_board_gray_erode, img_piece_gray_erode) for img_piece_gray_erode in img_pieces_gray_erode]

In [16]:
# Preprocessing
img_board = cv.imread('ChessBoard.png')
img_piece = cv.imread('WPawn.png')
board_x, board_y, color = img_board.shape
x_incrementer = board_x / 8
y_incrementer = board_y / 8

# Convert both images to grayscale
img_board_gray = cv.cvtColor(img_board, cv.COLOR_BGR2GRAY)
img_piece_gray = cv.cvtColor(img_piece, cv.COLOR_BGR2GRAY)

# morphological erosion for stabilization
s = 3
kernel = np.ones((s, s), np.uint8)
img_board_gray_erode = cv.erode(img_board_gray,kernel,iterations = 1)
img_piece_gray_erode = cv.erode(img_piece_gray,kernel,iterations = 1)

# All instances of matching templates
rects = find_template_multiple(img_board_gray_erode, img_piece_gray_erode)

# Matching color as given input
matching_color_list = [check_color(img_board_gray, img_piece_gray, r) for r in rects]

# Keep only matching color rectangles
matching_color_rects = [r for (r, is_matching) in zip(rects, matching_color_list) if is_matching]

piece_notation = display_matches(img_board, matching_color_rects, x_incrementer, y_incrementer)
print(piece_notation)

# Show the result
cv.imshow('Result', img_board)
cv.waitKey(0)
cv.destroyAllWindows()

[  38 1194  118  148]
[ 426 1194  118  148]
[ 814 1194  118  148]
[1202 1194  118  148]
[ 232 1194  118  148]
[ 620 1194  118  148]
[1008 1194  118  148]
[1396 1194  118  148]
['a8', 'c6', 'e4', 'g2', 'b7', 'd5', 'f3', 'h1']


CHANGE

In [3]:
# Load in all images being used
blackBishop = cv.imread('BBishop.png')
blackKing = cv.imread('BKing.png')
blackKnight = cv.imread('BKnight.png')
blackPawn = cv.imread('BPawn.png')
blackQueen = cv.imread('BQueen.png')
blackRook = cv.imread('BRook.png')

whiteBishop = cv.imread('WBishop.png')
whiteKing = cv.imread('WKing.png')
whiteKnight = cv.imread('WKnight.png')
whitePawn = cv.imread('WPawn.png')
whiteQueen = cv.imread('WQueen.png')
whiteRook = cv.imread('WRook.png')

img_board = cv.imread('ChessBoard.png')
img_piece = cv.imread('WRook.png')

img_pieces = [blackBishop, blackKing, blackKnight, blackPawn, blackQueen, blackRook,
              whiteBishop, whiteKing, whiteKnight, whitePawn, whiteQueen, whiteRook]

In [20]:
# Convert to grayscale
img_board_gray = cv.cvtColor(img_board, cv.COLOR_BGR2GRAY)
img_pieces_gray = [cv.cvtColor(img, cv.COLOR_BGR2GRAY) for img in img_pieces]

s = 3
kernel = np.ones((s,s), np.uint8)
# morphological gradient stabilizes the template matching by focusing on the shape's edges rather than its content.
img_board_gray_grad = cv.morphologyEx(img_board_gray, cv.MORPH_GRADIENT, kernel)
img_board_pieces_gray_grad = [cv.morphologyEx(img_gray, cv.MORPH_GRADIENT, kernel) for img_gray in img_pieces_gray]

In [28]:
# List of list of arrays of all pieces matching, 12 total in a chess opening (6 unique pairs)
rects = find_template_multiple(img_board_gray_erode, img_piece_gray_erode)
list_rects = [find_template_multiple(img_board_gray_grad, img_gray_pieces) for img_gray_pieces in img_board_pieces_gray_grad]


#matching_color_list = [check_color(img_board_gray, img_piece_gray, r) for r in rects]
print(matching_color_list)

# # Keep only matching color rectangles.
# matching_color_rects = [r for (r, is_matching) in zip(rects, matching_color_list) if is_matching]


# draw_results(img_board, matching_color_rects)

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'absdiff'
> Overload resolution failed:
>  - src1 is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src1'
